# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "The analysis should include statistical modeling and machine learning techniques to identify trends and predict market movements."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning techniques to predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yah

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy EV Deep Dive Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Calendars Trending Tickers Futures World Indices US Treasury Bonds Currencies Crypto Top ETFs Top Mutual Funds Highest Open Interest Highest Implied Volatility Stock Comparison Advanced Ch

Final Answer: 
Apple Inc. (AAPL) is currently trading at $189.87, with a slight increase of $0.03 (+0.02%). The market cap is at $2.911T, with a beta of 1.26 and a PE ratio of 29.48. The company's EPS is $6.44, and it has a forward dividend yield of 0.53%. AAPL's 1-year return is 10.93%, compared to the S&P 500's 29.04%. The 3-year return is 51.52%, and the 5-year return is 314.01%.

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products, including the iPhone, Mac, iPad, and wearables like AirPods and Apple Watch. They also provide services like Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Pay.

Overall, Apple Inc. has shown steady performance in the market with consistent growth over the years. It is a leader in the consumer electronics industry and continues to innovate and expand its product and service offerings.

> Finished chain.
 

Apple I

I need to delegate the task of analyzing the approved trading strategies for AAPL to determine the best execution methods to the Trading Strategy Developer.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Trading Strategy Developer",
    "task": "Analyze approved trading strategies for AAPL to determine the best execution methods",
    "context": "After analyzing the insights on AAPL stock performance, market trends, and the lack of specific user-defined risk tolerance information, I have developed a set of potential trading strategies for AAPL that align with a moderate risk tolerance. This includes long-term investment strategy, options trading strategy, sector rotation strategy, and technical analysis strategy."
}


> Entering new CrewAgentExecutor chain...
I need to gather more information on the approved trading strategies for AAPL in order to determine the best execution methods.

Action: Search the internet
Action Input: {"search_query": "AAPL approved tradi

I need to gather detailed risk analysis and mitigation strategies for the proposed trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Risk Advisor",
    "task": "Evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. Provide a detailed analysis of potential risks and suggest mitigation strategies.",
    "context": "After analyzing the insights on AAPL stock performance, market trends, and the lack of specific user-defined risk tolerance information, I have developed a set of potential trading strategies for AAPL that align with a moderate risk tolerance. The strategies include Long-Term Investment Strategy, Options Trading Strategy, Sector Rotation Strategy, and Technical Analysis Strategy."
}


> Entering new CrewAgentExecutor chain...
I need to thoroughly evaluate each proposed trading strategy for AAPL and assess the potential risks associated with them. I should also suggest mitigation strategi

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Introduction:
After analyzing the insights on AAPL stock performance, market trends, and the lack of specific user-defined risk tolerance information, a comprehensive risk analysis was conducted to evaluate the risks associated with the proposed trading strategies and execution plans for AAPL. The strategies include Long-Term Investment Strategy, Options Trading Strategy, Sector Rotation Strategy, and Technical Analysis Strategy.

Risk Factors Identified:
1. Legal Challenges: AAPL faces potential legal challenges related to intellectual property rights, patent disputes, and antitrust regulations. Legal battles can impact AAPL's stock price and market position.
2. Economic Conditions: Macroeconomic factors such as interest rates, inflation, and economic growth can influence AAPL's stock performance. Economic downturns or recessions may negatively impact AAPL's revenue and profitability.
3. Market Uncertainties: Volatility in the stock market, changes in investor sentiment, and geopolitical events can create uncertainties that affect AAPL's stock price and trading strategies.
4. Antitrust Issues: Regulatory scrutiny and antitrust investigations pose risks to AAPL's business operations and market competitiveness. Antitrust issues can lead to legal challenges and market instability.
5. U.S.-China Tensions: Trade tensions between the U.S. and China can impact AAPL's supply chain, production costs, and market access. Geopolitical risks may disrupt AAPL's global operations and revenue streams.

Mitigation Strategies:
1. Risk Monitoring: Implement a robust risk monitoring system to track legal, economic, market, and geopolitical risks that may affect AAPL's trading strategies. Stay informed about the latest developments and adjust the strategies accordingly.
2. Diversification: Diversify the investment portfolio across different asset classes, sectors, and regions to reduce concentration risk and enhance risk-adjusted returns. Spread the risk exposure to minimize potential losses.
3. Risk Management Techniques: Utilize risk management techniques such as stop-loss orders, hedging strategies, and position sizing to protect the investment portfolio from adverse market movements and unforeseen risks.
4. Contingency Planning: Develop contingency plans and alternative trading strategies to mitigate unexpected risks and market events. Be prepared to adapt to changing market conditions and adjust the strategies accordingly.

Conclusion:
The comprehensive risk analysis of the proposed trading strategies for AAPL highlights the importance of identifying potential risks, developing mitigation strategies, and implementing risk management techniques to protect the investment portfolio and optimize trading performance. By closely monitoring risk factors, diversifying the trading strategies, and proactively managing risks, investors can navigate market uncertainties and capitalize on opportunities while aligning with a moderate risk tolerance. Continuous evaluation and adjustment of the strategies are essential to achieve long-term trading success and sustainable returns.